In [1]:
#Import the modules
import pandas as pd
pd.set_option('display.max_columns', None)
# file paths to files
#players
path_2015 = "raw/players_15.csv"
path_2016 = "raw/players_16.csv"
path_2017 = "raw/players_17.csv"
path_2018 = "raw/players_18.csv"
path_2019 = "raw/players_19.csv"
path_2020 = "raw/players_20.csv"
paths = [path_2015, path_2016,path_2017,path_2018,path_2019,path_2020]
#teams
path_teams = "raw/big_five_1995-2019.csv"

In [2]:
#read the team file to focus on the 3 leagues we want to analyse
teams = pd.read_csv(path_teams)

#find the 3 leagues for 2015 to 2019
country_code = teams["Country"].unique()
print(country_code)
years = [2015,2016,2017,2018,2019]
teams = teams[(teams["Country"].isin(["ENG","ESP","IT"]))& (
                    teams["Year"].isin(years))]

teams = teams[['Team 1','Team 2', 'Year', 'Country',"FT Team 1","FT Team 2",'Team 1 (pts)', 'Team 2 (pts)']]

eng_teams = teams[teams["Country"]== "ENG"]
it_teams = teams[teams["Country"]== "IT"]
spain_teams = teams[teams["Country"]== "ESP"]

def rankingteams(teams):
    ranking = {}
    for year in years:
        a = teams[teams["Year"]==year]
        list = a["Team 1"].unique()
        points = [0]*len(list)
        goalsp = [0]*len(list)
        goalsn = [0]*len(list)
        for index,team in enumerate(list):
            points[index] += a[a["Team 1"]==team]["Team 1 (pts)"].sum()
            points[index] += a[a["Team 2"]==team]["Team 2 (pts)"].sum()
            goalsp[index] += a[a["Team 1"]==team]["FT Team 1"].sum()
            goalsn[index] += a[a["Team 1"]==team]["FT Team 2"].sum()
            ranking.setdefault(f"Year",[]).append(year)
            ranking.setdefault(f"Teams",[]).append(team)
            ranking.setdefault(f"Points",[]).append(points[index])
            ranking.setdefault(f"Goals+",[]).append(goalsp[index])
            ranking.setdefault(f"Goals-",[]).append(goalsn[index])
    return pd.DataFrame(ranking)
ranking_it_df = rankingteams(it_teams)
ranking_eng_df = rankingteams(eng_teams)
ranking_spain_df = rankingteams(spain_teams)
# Ranking england and some preparation to match the teams in the other dataframes
ranking_eng_df["Teams"] = ranking_eng_df["Teams"].str.replace("\s$","")
ranking_eng_df["Teams"] = ranking_eng_df["Teams"].str.replace("\s+FC", "")
ranking_eng_df["Teams"] = ranking_eng_df["Teams"].str.replace("\s+AFC", "")
ranking_eng_df["Teams"] = ranking_eng_df["Teams"].str.replace("^AFC\s", "")
ranking_eng_df.sort_values(["Year","Points"], ascending = False).head()

['ENG' 'ESP' 'FR' 'GER' 'IT']


,Year,Teams,Points,Goals+,Goals-
80,2019,Liverpool,82,40,12
96,2019,Manchester City,57,33,12
87,2019,Leicester City,53,30,15
98,2019,Chelsea,48,22,15
89,2019,Manchester United,45,29,12


In [3]:
#ranking Italy, clean and export
ranking_it_df["Teams"] = ranking_it_df["Teams"].str.replace("^\s?","")
ranking_it_df["Teams"] = ranking_it_df["Teams"].str.replace("\s$","")
ranking_it_df["Teams"] = ranking_it_df["Teams"].str.replace("\s+FC", "")
ranking_it_df["Teams"] = ranking_it_df["Teams"].str.replace("\s+BC", "")
ranking_it_df["Teams"] = ranking_it_df["Teams"].str.replace("\s+CFC", "")
ranking_it_df["Teams"] = ranking_it_df["Teams"].str.replace("^\s?UC", "")
ranking_it_df["Teams"] = ranking_it_df["Teams"].str.replace("^\s?US", "")
ranking_it_df["Teams"] = ranking_it_df["Teams"].str.replace("^\s?SSC\s?", "")
ranking_it_df.sort_values(["Year","Points"], ascending = False).head()

,Year,Teams,Points,Goals+,Goals-
92,2019,Juventus,63,31,10
93,2019,SS Lazio,62,39,10
89,2019,FC Internazionale Milano,54,23,10
94,2019,Atalanta,48,33,19
84,2019,AS Roma,45,26,19


In [8]:
#ranking Spain
ranking_spain_df["Teams"] = ranking_spain_df["Teams"].str.replace("\s$","")
ranking_spain_df.sort_values(["Year","Points"], ascending = False).head()

,Year,Teams,Points,Goals+,Goals-
95,2019,FC Barcelona,58,45,12
93,2019,Real Madrid,56,27,9
96,2019,Sevilla FC,47,19,13
94,2019,Getafe CF,46,21,13
97,2019,Real Sociedad,46,27,12


In [9]:
def playersfunc(filepath,year):
    players_df = pd.read_csv(filepath)
    players_df = players_df[['short_name','age','height_cm','weight_kg','nationality','club','value_eur','overall']]
    players_df = players_df.rename({"club":"Teams"},axis=1)
    f = {"age":"mean", "height_cm":"mean","weight_kg":"mean","overall":"mean","value_eur":"mean"}
    players_df = players_df.groupby("Teams").agg(f)
    players_df["Year"]= year
    players_df["Year"] = players_df["Year"].astype('int')
    players_df = players_df.reset_index()
    return players_df
def cleanup(df):
    df["Teams"] = df["Teams"].str.replace("RC Celta", "RC Celta Vigo")
    df["Teams"] = df["Teams"].str.replace("Real Sporting de Gijón", "Sporting Gijón")
    df["Teams"] = df["Teams"].str.replace("Athletic Club de Bilbao", "Athletic Club Bilbao")
    df["Teams"] = df["Teams"].str.replace("Deportivo de La Coruña", "RCD La Coruña")
    df["Teams"] = df["Teams"].str.replace("Milan", "AC Milan")
    df["Teams"] = df["Teams"].str.replace("Lazio", "SS Lazio")
    df["Teams"] = df["Teams"].str.replace("Fiorentina", "ACF Fiorentina")
    df["Teams"] = df["Teams"].str.replace("Inter", "FC Internazionale Milano")
    df["Teams"] = df["Teams"].str.replace("Sassuolo", "Sassuolo Calcio")
    df["Teams"] = df["Teams"].str.replace("Chievo Verona", "AC Chievo Verona")
    df["Teams"] = df["Teams"].str.replace("Roma", "AS Roma")
    df["Teams"] = df["Teams"].str.replace("Udinese", "Udinese Calcio")
    df["Teams"] = df["Teams"].str.replace("Cagliari", "Cagliari Calcio")
    df["Teams"] = df["Teams"].str.replace("Crotone", "FC Crotone")
    return df

In [10]:
players_df_2015 = playersfunc(path_2015,2015)
players_df_2015 = cleanup(players_df_2015)
players_df_2016 = playersfunc(path_2016,2016)
players_df_2016 = cleanup(players_df_2016)
players_df_2017 = playersfunc(path_2017,2017)
players_df_2017 = cleanup(players_df_2017)
players_df_2018 = playersfunc(path_2018,2018)
players_df_2018 = cleanup(players_df_2018)
players_df_2019 = playersfunc(path_2019,2019)
players_df_2019 = cleanup(players_df_2019)

In [11]:
#append the lists to have 1 data frame
players_data = players_df_2015.append([players_df_2016,players_df_2017,players_df_2018,players_df_2019])

In [12]:
#merging with the 3 different competitions
eng_df = ranking_eng_df.merge(players_data,on=["Year","Teams"],how = "left")
it_df = ranking_it_df.merge(players_data,on=["Year","Teams"],how = "left")
spain_df = ranking_spain_df.merge(players_data,on=["Year","Teams"],how = "left")
#update the latest data
eng_df.iloc[-20:,2] = [99,39,34,54,43,34,59,62,44,66,56,35,41,49,21,52,81,54,66,59]
eng_df.iloc[-20:,3] = [85,49,40,43,31,36,61,67,38,66,56,41,39,44,26,51,102,39,69,51]
eng_df.iloc[-20:,4] = [33,62,65,50,50,64,47,41,58,36,48,67,54,56,75,60,35,39,54,40]
spain_df.iloc[-20:,2] = [51,37,53,33,36,60,39,25,41,70,56,49,52,87,54,82,70,56,42,42]
spain_df.iloc[-20:,3] = [41,37,46,40,30,63,34,27,48,51,52,47,46,70,43,86,54,56,39,32]
spain_df.iloc[-20:,4] = [38,49,53,65,51,49,59,59,60,27,45,53,54,25,37,38,34,48,56,43]
it_df.iloc[-20:,2] = [49,49,45,45,70,42,20,40,49,82,47,66,83,78,78,39,35,51,62,25]
it_df.iloc[-20:,3] = [56,51,37,52,77,48,27,46,47,81,52,63,76,79,98,47,52,69,61,35]
it_df.iloc[-20:,4] = [57,48,51,56,51,65,70,68,51,36,65,46,43,42,48,73,85,63,50,79]

In [18]:
#rounding the columns to make it more readable
list_df = [eng_df,spain_df,it_df]
for df in list_df:
    df["age"] = df["age"].map(lambda x: round(x,2))
    df["height_cm"] = df["height_cm"].map(lambda x: round(x,2))
    df["weight_kg"] = df["weight_kg"].map(lambda x: round(x,2))
    df["overall"] = df["overall"].map(lambda x: round(x,2))
    df["value_eur"] = df["value_eur"].map(lambda x: round(x))
    df = df.rename(columns={"age": "avg age","height_cm":"avg heights (cm)","weight_kg":"avg weight (kg)","overall":"agv player rating","value_eur": "avg player value (EU)"})

KeyError: 'age'

In [ ]:
#export to csv
spain_df.to_csv("cleaned/spainranking.csv")
it_df.to_csv("cleaned/itranking.csv")
eng_df.to_csv("cleaned/engranking.csv")

In [16]:
#diversity
eng_df

,Year,Teams,Points,Goals+,Goals-,avg age,avg heights (cm),avg weight (kg),agv player rating,avg player value (EU)
0,2015,Bournemouth,42,23,34,25.03,180.63,75.97,63.93,0
1,2015,Chelsea,50,32,30,23.76,183.42,77.30,73.27,0
2,2015,Everton,47,35,30,25.33,180.67,75.55,72.36,0
3,2015,Leicester City,81,35,18,25.03,181.85,75.76,66.45,0
4,2015,Manchester United,66,27,9,23.94,183.00,75.45,73.39,0
...,...,...,...,...,...,...,...,...,...,...
95,2019,Southampton,52,51,60,23.73,183.39,75.79,71.42,5977121
96,2019,Manchester City,81,102,35,24.09,180.45,74.12,76.85,22951364
97,2019,Sheffield United,54,39,39,25.60,182.03,75.13,66.27,1438500
98,2019,Chelsea,66,69,54,25.21,182.82,77.00,76.79,18798788
